# Cerebellar Model Articulation Controller (CMAC)

This is a proposed computing model based on the principles of the brain's architecture. This is similar in inspiration to multilayer perceptrons (aka. MLPs, neural networks).

In a similar way to MLPs, the model can be defined as [1]:

$$f : S \rightarrow A$$
$$g : A \rightarrow P$$

and $$y = \sum A^{\*}_i w_i$$

Where $A$ are association cells (TODO explain), $P$ are response output vectors, $S$ are the sensory input vectors.

### MLP vs CMAC

The difference between MLPs and CMAC [1] is that in MLPs $f$ is fully connected, while CMAC restricts the associations to neighbors. This is more in tune with Convolutional Nueral Networks in a sense (TODO cite).

### Differences from the human brain

There are suggestions that the cerebellum is capable of exclusive disjunction which is not in CMAC [1]. TODO more on this.

### Benefits of CMAC

"the restriction on mapping function effectively decreased the
chance of been trapped in a local minimum during learning
process" [1]

TODO: more on this

### Disadvantages

TODO

### Basic architecture



## References

[1] Xing, Frank Z. "A Historical Review of Forty Years of Research on CMAC." arXiv preprint arXiv:1702.02277 (2017).